In [3]:
import numpy as np
import pandas as pd

In [ ]:
X_train = pd.read_csv('housing_dataset/housing_X_train.csv', header=None).values.T
y_train = pd.read_csv('housing_dataset/housing_y_train.csv', header=None).values.ravel()
X_test = pd.read_csv('housing_dataset/housing_X_test.csv', header=None).values.T
y_test = pd.read_csv('housing_dataset/housing_y_test.csv', header=None).values.ravel()

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train_norm = sc.fit_transform(X_train)
X_test_norm = sc.transform(X_test)

# def normalize(X):
#     mean = np.mean(X, axis=0)
#     std = np.std(X, axis=0)
#     return (X - mean) / std

# X_train_norm = normalize(X_train)
# X_test_norm = normalize(X_test)

# Ridge regression algorithm using the closed form solution

In [48]:
def ridge_regression(X, y, lambda_):
    
    n_samples, n_features = X.shape
    
    I = np.identity(n_features)
    
    w = np.linalg.solve(X.T @ X + lambda_ * I, X.T @ y)
    return w

def compute_mse(y_true, y_pred):
    """Compute Mean Squared Error."""
    return np.mean((y_true - y_pred) ** 2)

lambdas = [0, 0.25, 0.5, 0.75, 1]

for lambda_ in lambdas:
    weights = ridge_regression(X_train_norm, y_train, lambda_)
    
    # Predict on the training set
    y_train_pred = X_train_norm @ weights
    train_mse = compute_mse(y_train, y_train_pred)
    
    # Predict on the test set
    y_test_pred = X_test_norm @ weights
    test_mse = compute_mse(y_test, y_test_pred)
    
    print(f"Lambda: {lambda_}, Training MSE: {train_mse}, Test MSE: {test_mse:.2f}")


Lambda: 0, Training MSE: 662.1461594616705, Test MSE: 576.25
Lambda: 0.25, Training MSE: 662.1462363552637, Test MSE: 575.39
Lambda: 0.5, Training MSE: 662.1464647289267, Test MSE: 574.55
Lambda: 0.75, Training MSE: 662.1468411939754, Test MSE: 573.75
Lambda: 1, Training MSE: 662.1473624551902, Test MSE: 572.96


# Ridge regression using the gradient descent

In [49]:
def ridge_regression_gradient_descent(X, y, lambda_, learning_rate=0.1, n_iterations=1000):
    n_samples, n_features = X.shape
    w = np.zeros(n_features)
    
    for _ in range(n_iterations):
        y_pred = X @ w
        gradient = (X.T @ (y_pred - y)) / n_samples + lambda_ * w

        w -= learning_rate * gradient
    
    return w

lambdas = [0, 0.25, 0.5, 0.75, 1]
learning_rate = 0.01
n_epochs = 1000

for lambda_ in lambdas:
    weights = ridge_regression_gradient_descent(X_train_norm, y_train, lambda_, learning_rate, n_epochs)
    # print(weights)
    # Predict on the training set
    y_train_pred = X_train_norm @ weights
    train_mse = compute_mse(y_train, y_train_pred)
    
    # Predict on the test set
    y_test_pred = X_test_norm @ weights
    test_mse = compute_mse(y_test, y_test_pred)
    
    # Report results
    print(f"Lambda: {lambda_}, Training MSE: {train_mse:.2f}, Test MSE: {test_mse:.2f}")

[ 0.66252605  0.35681172  0.09909461  0.21276072 -0.56303703  6.13476605
 -1.10171694 -1.75664123  0.17276305 -1.04377504 -1.36794469  0.71894249
 -1.12162724]
Lambda: 0, Training MSE: 662.23, Test MSE: 563.22
[ 0.26755566  0.38471732 -0.23143633  0.27400512 -0.23076865  4.58775408
 -0.55088491 -1.2175447   0.17141993 -0.88662828 -1.37975465  0.58743096
 -1.71966582]
Lambda: 0.25, Training MSE: 664.26, Test MSE: 585.79
[ 0.11485842  0.39428857 -0.35905161  0.28141285 -0.15857259  3.79672506
 -0.39567312 -0.93667734  0.17309963 -0.78274482 -1.31900103  0.50950701
 -1.7668437 ]
Lambda: 0.5, Training MSE: 667.03, Test MSE: 610.77
[ 0.03635506  0.3966007  -0.415408    0.27276699 -0.14420778  3.28642842
 -0.33864083 -0.75396009  0.17023149 -0.70719712 -1.24097225  0.45612618
 -1.69754911]
Lambda: 0.75, Training MSE: 669.84, Test MSE: 620.40
[-0.01119932  0.39460861 -0.44069561  0.25901849 -0.14501816  2.91693149
 -0.31266734 -0.62279004  0.1643112  -0.64867449 -1.1626942   0.41643266
 -1.60

# Lasso regression using gradient descent

In [50]:
def lasso_regression_gradient_descent(X, y, lambda_, learning_rate=0.1, n_epochs=1000):
    n_samples, n_features = X.shape
    w = np.zeros(n_features)

    for _ in range(n_epochs):
        y_pred = X @ w
        gradient = (X.T @ (y_pred - y)) / n_samples
        
        # lasso gradient with subgradient for L1 regularization
        w_sign = np.sign(w)
        gradient += lambda_ * w_sign
        
        w -= learning_rate * gradient

    return w

for lambda_ in lambdas:
    weights = lasso_regression_gradient_descent(X_train_norm, y_train, lambda_, learning_rate, n_epochs)
    # print(weights)
    y_train_pred = X_train_norm @ weights
    train_mse = compute_mse(y_train, y_train_pred)
    
    # Predict on the test set
    y_test_pred = X_test_norm @ weights
    test_mse = compute_mse(y_test, y_test_pred)
    
    # Report results
    print(f"Lambda: {lambda_}, Training MSE: {train_mse:.2f}, Test MSE: {test_mse:.2f}")

[ 0.66252605  0.35681172  0.09909461  0.21276072 -0.56303703  6.13476605
 -1.10171694 -1.75664123  0.17276305 -1.04377504 -1.36794469  0.71894249
 -1.12162724]
Lambda: 0, Training MSE: 662.23, Test MSE: 563.22
[ 3.38669477e-03  3.59870653e-03 -2.74228251e-03  5.13639841e-02
 -5.51339923e-04  6.16867548e+00 -5.04050030e-01 -7.11446292e-01
 -7.47973686e-04 -7.02301054e-01 -1.31000888e+00  3.52037469e-01
 -1.09402803e+00]
Lambda: 0.25, Training MSE: 663.12, Test MSE: 741.39
[ 1.63323814e-03  1.45121636e-03 -1.79798684e-03  3.25233421e-03
  3.63504841e-03  6.15748816e+00 -6.15297151e-03 -1.31064158e-02
  1.46601498e-03 -5.05093179e-01 -1.11007863e+00  1.00149240e-01
 -1.02584322e+00]
Lambda: 0.5, Training MSE: 664.58, Test MSE: 685.37
[ 4.05270942e-04 -2.71636698e-03  4.21528233e-03  6.67658930e-03
  1.41927497e-03  6.12293634e+00 -1.05533836e-02 -1.00715095e-02
 -2.04961961e-03 -3.18211766e-01 -9.00919659e-01  1.08463024e-02
 -9.03076318e-01]
Lambda: 0.75, Training MSE: 665.37, Test MSE: 